In [1]:
import tweepy
import pandas as pd
import ast
import nltk
from nltk.corpus import brown
from nltk import word_tokenize
from tqdm import tqdm
import time
from nltk.probability import FreqDist
import math
import warnings;
warnings.filterwarnings('ignore');

In [2]:
#Preprocessing
# read date
df = pd.read_csv('tweets_message_tokyo.csv', encoding = "utf-8")
# drop rows which has at least one element to be Nan(drop 63562 here)
df = df.dropna()
#convert text to utf8
df['text'] = df['text'].apply(lambda x: ast.literal_eval(x).decode('utf8'))
# #sort df based on time
# df_sort = df.sort_values(by=['time'])
# df_sort

In [3]:
from pathlib import Path 
#save data in dictionary(including dates from '2021-7-19' to '2021-8-11')
date_range = pd.date_range(start='2021-07-19', end='2021-08-11')

## Read CSV files back to dataframe dict
df_dict = {}
for i in date_range:
    date = i.strftime('%Y-%m-%d')
    filepath = Path('dataframes_by_date/'+date+".csv")
    df_dict[date] = pd.read_csv(filepath, encoding = "utf-8")
df_dict["2021-07-31"]

,Unnamed: 0,id,time,text,city_name,coordinates,place
0,39424,1421259330838540295,2021-07-31 00:00:04+00:00,#TeamGB won #triathlon #Olympics2020 #Toyko2020,"West Chester, OH","[[[-84.4832977, 39.2957423], [-84.346523, 39.2...",Place(_api=<tweepy.api.API object at 0x2b23538...
1,17157,1421259710884417537,2021-07-31 00:01:35+00:00,Opposite ends of the athletic body type @laury...,"Johnson City, TN","[[[-82.4759055, 36.2592231], [-82.300523, 36.2...",Place(_api=<tweepy.api.API object at 0x2b23538...
2,39420,1421259737694445583,2021-07-31 00:01:41+00:00,@AnglingNArchery @USABaseball @TeamUSA @WBSC @...,"Zanesville, OH","[[[-82.121534, 39.908312], [-81.976901, 39.908...",Place(_api=<tweepy.api.API object at 0x2b23538...
3,30184,1421259768891576321,2021-07-31 00:01:49+00:00,It’s the Olympics. There are no weekends. See ...,"New Canaan, CT","[[[-73.5537, 41.11369], [-73.4489288, 41.11369...",Place(_api=<tweepy.api.API object at 0x2b23538...
4,54236,1421259812097171461,2021-07-31 00:01:59+00:00,@Timothy58100352 @USABaseball @TeamUSA @WBSC @...,"Zanesville, OH","[[[-82.121534, 39.908312], [-81.976901, 39.908...",Place(_api=<tweepy.api.API object at 0x2b23538...
...,...,...,...,...,...,...,...
2863,1656,1421621097166901248,2021-07-31 23:57:36+00:00,This was team USA where you get team lebron fr...,"Nevada, USA","[[[-120.00574, 35.002086], [-114.039649, 35.00...",Place(_api=<tweepy.api.API object at 0x2b23538...
2864,2139,1421621301656043520,2021-07-31 23:58:25+00:00,@ASoftstar If that's the human standard...no w...,"Minneapolis, MN","[[[-93.329515, 44.889964], [-93.194578, 44.889...",Place(_api=<tweepy.api.API object at 0x2b23538...
2865,6779,1421621505885188098,2021-07-31 23:59:14+00:00,Way to rep Ohio 🙌🇺🇸🥇🥈🥉🎖 https://t.co/KmgqPJVahc,"Canal Fulton, OH","[[[-81.623096, 40.842646], [-81.536917, 40.842...",Place(_api=<tweepy.api.API object at 0x2b23538...
2866,1395,1421621510578581507,2021-07-31 23:59:15+00:00,I’m loving the guys calling the men’s @usavoll...,"Texas, USA","[[[-106.645646, 25.837092], [-93.508131, 25.83...",Place(_api=<tweepy.api.API object at 0x2b23538...


In [4]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
us_state_to_abbrev["Texas"]

'TX'

In [5]:
import numpy
def append_states(date):
    cities_list = df['city_name']
    states_list = []
    for city in cities_list:
        city = city.split(', ')
        if len(city) <= 1:
            state = numpy.nan
        elif(city[1]=="USA"):
            state = city[0]
        else:
            state = city[1]
        if(state in us_state_to_abbrev.keys()):
            state = us_state_to_abbrev[state]
        states_list.append(state)
    df['state'] = states_list

In [6]:
for i in date_range:
    append_states(i.strftime('%Y-%m-%d'))
df 

,id,time,text,city_name,coordinates,place,state
0,1419808468324790274,2021-07-26 23:54:52+00:00,I hadn’t seen you for awhile… I thought you we...,"Newport, KY","[[[-84.504476, 39.064988], [-84.4611, 39.06498...",Place(_api=<tweepy.api.API object at 0x2b23538...,KY
1,1419343586320019458,2021-07-25 17:07:35+00:00,There’s a bad joke here somewhere https://t.co...,"Inglewood, CA","[[[-118.378887, 33.9253169], [-118.3134177, 33...",Place(_api=<tweepy.api.API object at 0x2b23538...,CA
2,1419372964823408640,2021-07-25 19:04:20+00:00,The #Olympics seem to be starting out slowly f...,"Central City, KY","[[[-87.154448, 37.269115], [-87.0936138, 37.26...",Place(_api=<tweepy.api.API object at 0x2b23538...,KY
3,1425093660165738509,2021-08-10 13:56:20+00:00,https://t.co/u0G6OPzAav,"Gainesville, VA","[[[-77.675429, 38.763188], [-77.579294, 38.763...",Place(_api=<tweepy.api.API object at 0x2b23538...,VA
4,1419343311375151107,2021-07-25 17:06:30+00:00,@SpencerBoyd Olympics/Baseball,"Vicksburg, MI","[[[-85.583959, 42.089785], [-85.509529, 42.089...",Place(_api=<tweepy.api.API object at 0x2b23538...,MI
...,...,...,...,...,...,...,...
63558,1424501549338873860,2021-08-08 22:43:29+00:00,Please do yourself a huge favor a tap in the G...,"Spartanburg, SC","[[[-82.045319, 34.875556], [-81.8352774, 34.87...",Place(_api=<tweepy.api.API object at 0x2b23538...,SC
63559,1424503920743780356,2021-08-08 22:52:55+00:00,Yes! https://t.co/VOMo71tqbd,"Davis, CA","[[[-121.803252, 38.526843], [-121.675074, 38.5...",Place(_api=<tweepy.api.API object at 0x2b23538...,CA
63560,1424504989922299915,2021-08-08 22:57:10+00:00,https://t.co/ApG332Hp0Q,"Mehlville, MO","[[[-90.3557179, 38.477027], [-90.274797, 38.47...",Place(_api=<tweepy.api.API object at 0x2b23538...,MO
63561,1421637224005869569,2021-08-01 01:01:41+00:00,🤣🤣🤣 https://t.co/4JSLEKvfl5,"Florida, USA","[[[-87.634643, 24.396308], [-79.974307, 24.396...",Place(_api=<tweepy.api.API object at 0x2b23538...,FL


In [7]:
df_sort_state = df.sort_values(by=['state'])

In [8]:
df_sort_state = df_sort_state.dropna()
df_sort_state

,id,time,text,city_name,coordinates,place,state
46474,1420061516938694661,2021-07-27 16:40:23+00:00,So proud of you ❤️ https://t.co/69UOSVouRb,"Kodiak, AK","[[[-152.455159, 57.771608], [-152.325453, 57.7...",Place(_api=<tweepy.api.API object at 0x2b23538...,AK
53404,1419849470930419717,2021-07-27 02:37:48+00:00,Super cool experience watching this happen whi...,"Anchorage, AK","[[[-150.0352589, 61.043884], [-149.48233, 61.0...",Place(_api=<tweepy.api.API object at 0x2b23538...,AK
39615,1422629411791151107,2021-08-03 18:44:17+00:00,Me encanta la emoción del comentarista 🥳😍 http...,"Fairbanks, AK","[[[-147.81382, 64.810474], [-147.543503, 64.81...",Place(_api=<tweepy.api.API object at 0x2b23538...,AK
52304,1419854239925374982,2021-07-27 02:56:45+00:00,One for all Alaskans 🙌🙌🙌 https://t.co/H3RGbwo2h4,"Juneau, AK","[[[-134.6678953, 58.260245], [-134.3499372, 58...",Place(_api=<tweepy.api.API object at 0x2b23538...,AK
39596,1419910465040261122,2021-07-27 06:40:10+00:00,Watched this live while at a bar in downtown A...,"Anchorage, AK","[[[-150.0352589, 61.043884], [-149.48233, 61.0...",Place(_api=<tweepy.api.API object at 0x2b23538...,AK
...,...,...,...,...,...,...,...
37861,1420225565315919875,2021-07-28 03:32:15+00:00,Listening to @DanHicksNBC and @RowdyGaines cal...,"Cheyenne, WY","[[[-104.8609088, 41.078217], [-104.713275, 41....",Place(_api=<tweepy.api.API object at 0x2b23538...,WY
10186,1418991050987905031,2021-07-24 17:46:44+00:00,I want to see more of the men playing beach vo...,"Wheatland, WY","[[[-104.982619, 42.030885], [-104.9248, 42.030...",Place(_api=<tweepy.api.API object at 0x2b23538...,WY
59275,1422720954073903110,2021-08-04 00:48:02+00:00,Do the sprinters really conserve that much ene...,"Cheyenne, WY","[[[-104.8609088, 41.078217], [-104.713275, 41....",Place(_api=<tweepy.api.API object at 0x2b23538...,WY
45788,1420934206041907204,2021-07-30 02:28:09+00:00,I can’t stop thinking that the safety pinned p...,"Casper, WY","[[[-106.4322, 42.791368], [-106.229362, 42.791...",Place(_api=<tweepy.api.API object at 0x2b23538...,WY


In [9]:
df_state_group = df_sort_state.groupby('state')['text'].apply(' '.join).reset_index()
df_state_group[df_state_group["state"]=="TX"]

,state,text
71,TX,Tarde pero con sueño .. venga #Mexico !!! Con ...


In [10]:
df_state_group[df_state_group["state"]=="TX"]["text"]

71    Tarde pero con sueño .. venga #Mexico !!! Con ...
Name: text, dtype: object

In [11]:
df_state_group.head(5)

,state,text
0,AK,So proud of you ❤️ https://t.co/69UOSVouRb Sup...
1,AL,Wowza! 😮 https://t.co/vVoW0m85bS 🙋🏾‍♀️ https:/...
2,AR,Can’t help but LOL 🤣 https://t.co/kfud7Kejmv F...
3,AZ,"As coverage of #Tokyo2020 Olympics continues, ..."
4,Atlanta,In honor of the #Olympics I’m going for gold a...


In [12]:
df_state_group[df_state_group["state"]=="AK"]["text"]

0    So proud of you ❤️ https://t.co/69UOSVouRb Sup...
Name: text, dtype: object

In [ ]:
df_state_dict = {}
state_range = df_sort_state["state"].tolist()
new_state_range = []
prev_place =""

p = tqdm(total=len(state_range), disable=False)
for i in tqdm(range(len(state_range)), position=0, leave=True):
    place = state_range[i]
    if place not in us_state_to_abbrev.values():
        continue
    if(prev_place!=place):
        prev_place=place
        new_state_range.append(place)
    df_state_dict[place] = df_state_group[(df_state_group['state']==place)]
    time.sleep(0.01)
    p.update(1)
    tqdm._instances.clear()

 15%|█████▋                                | 9500/62915 [01:41<09:30, 93.69it/s]

In [ ]:
df_state_dict["TX"]['text'].astype(str)

In [ ]:
#calculate topic freq
# document frequency: in how many files of the Brown corpus does each word appear
brown.fileids()
stopwords = nltk.corpus.stopwords.words("english")
stopwords += ["rt", "got", "thats", "would", "going", "u", 
               "get", "also", "one", "could", "said", "like",
               "via", "say", "amp", "many", "need", "day", "want", 
               "last", "people", "done", "two",
               "outbreak", "know", "upon", "may", "way",
               "saw", "gtgt", "olympics", "tokyo2020", "tokyo", "olympics2021", "usa"
               "olympics2020", "@olympics", "tokyoolympics", "olympic", "https"]

docfreq = { }

for brown_part in brown.fileids():
    fd = nltk.FreqDist(brown.words(fileids = brown_part))
    
    for word in fd.keys():
        word = word.lower()
        
        if word in stopwords:
            continue
        if not word.isalnum():
            continue
            
        if word in docfreq:
            docfreq[word] = docfreq[word] + 1
        else:
            docfreq[word] = 1

In [ ]:
def compute_freq_dict(sentence):
    freq_example = FreqDist()
    words = word_tokenize(sentence)
    for word in words:
        word = word.lower()

        if word in stopwords:
            continue
        if not word.isalnum():
            continue
        freq_example[word.lower()] += 1 
    return freq_example

In [ ]:
word_freq_all=FreqDist()
tweets = df_state_dict["TX"]['text'].astype(str)
for sentence in tweets:
    word_freq = compute_freq_dict(sentence)
    word_freq_all = word_freq_all+word_freq
word_freq_all

In [ ]:
word_freq_all.most_common(3)

In [ ]:
word_freq_state = {}
word_freq_state["TX"] = word_freq_all

In [ ]:
word_freq_state["TX"].most_common(3)

In [ ]:
def append_word_freq(state, word_freq_state):
    word_freq_all=FreqDist()
    tweets = df_state_dict[state]['text'].astype(str)
    for sentence in tweets:
        word_freq = compute_freq_dict(sentence)
        word_freq_all = word_freq_all+word_freq
    word_freq_state[state] = word_freq_all

In [ ]:
len(new_state_range)

In [ ]:
word_freq_state = {}
tqdm._instances.clear()
p = tqdm(total=len(new_state_range), disable=False)
for i in tqdm(range(len(new_state_range)), position=0, leave=True):
    state = new_state_range[i]
    append_word_freq(state, word_freq_state)
    time.sleep(0.01)
    p.update(1)
tqdm._instances.clear()
p.close()

In [ ]:
word_freq_state["TX"].most_common(3)

In [ ]:
pd.DataFrame.from_dict(word_freq_state["TX"], orient='index')

In [ ]:
hot_topics = ["covid", "cancel", "softball", "openingceremony", "skateboarding", "basketball", 
              "swimming", "simone", "gold", "marathon", "volleyball", "closing"]

In [ ]:
hot_topics.append("state")

In [ ]:
hot_topics

In [ ]:
def make_dict_hot_freq_state(state):
    hot_freq_state = {}
    hot_word_freq = {}
    for topic in hot_topics:
        hot_word_freq[topic] = word_freq_state[state][topic]
    hot_word_freq["state"] = state
    hot_freq_state[state] = hot_word_freq
    return hot_freq_state[state]

In [ ]:
import pandas as pd
hot_freq_state = {}
df_hot_freq = pd.DataFrame()

In [ ]:
alllist=[]

In [ ]:
for state in new_state_range:
    hot_freq_state[state] = make_dict_hot_freq_state(state)
    pd_n = pd.DataFrame.from_dict(hot_freq_state[state], orient='index').T
    alllist.append(pd_n)

In [ ]:
df_hot_freq=pd.concat(alllist)

In [ ]:
df_hot_freq.head(5)

In [ ]:
word_freq_state["TX"]["gold"]

In [ ]:
def plot_topic_mention(topic):
    freq_list = [word_freq_state[state][topic] for state in new_state_range]
    # import plotly.express as px
    fig = px.line(x=new_state_range, y=freq_list, title=topic+' mention frequnecy by state'
                 ,labels=dict(x="state", y= topic+" Mention Times", color="Place"))
    fig.show()  

In [ ]:
import plotly.express as px
hot_topics = ["covid", "cancel", "softball", "openingceremony", "skateboarding", "basketball", 
              "swimming", "simone", "gold", "marathon", "volleyball", "closing"]
for topic in hot_topics:
    plot_topic_mention(topic)

In [ ]:
df_hot_freq.head(5)

In [ ]:
df_state_dict["TX"]["text"]

In [ ]:
def compute_tf_idf(sentence):
    dfidf = FreqDist()
    num_brown_parts = len(list(brown.fileids()))
    words = word_tokenize(sentence)
    tf_dict = compute_freq_dict(sentence)
    for word in words:
        word = word.lower()

        if word in stopwords:
            continue
        if not word.isalnum():
            continue
        if word not in docfreq:
            continue
        df = docfreq[word]
        idf = math.log(num_brown_parts / df )
        dfidf[word] = (tf_dict[word]/len(sentence))*idf
    return dfidf
        
# def append_word_freq(state, word_freq_state):
#     word_freq_all=FreqDist()
#     tweets = df_state_dict[state]['text'].astype(str)
#     for sentence in tweets:
#         word_freq = compute_freq_dict(sentence)
#         word_freq_all = word_freq_all+word_freq
#     word_freq_state[state] = word_freq_all

def append_tf_idf_freq(state, tf_idf_freq_state):
    word_tf_idf_all=FreqDist()
    tweets = df_state_dict[state]['text'].astype(str)
    for sentence in tweets:
        word_tf_idf = compute_tf_idf(sentence)
        word_tf_idf_all = word_tf_idf_all+word_tf_idf
    tf_idf_freq_state[state] =  word_tf_idf

In [ ]:
word_tfidf_state = {}
tqdm._instances.clear()
p = tqdm(total=len(new_state_range), disable=False)
for i in tqdm(range(len(new_state_range)), position=0, leave=True):
    state = new_state_range[i]
    append_tf_idf_freq(state, word_tfidf_state)
    time.sleep(0.01)
    p.update(1)
tqdm._instances.clear()
p.close()

In [ ]:
word_freq_state["TX"]["gold"]

In [ ]:
word_tfidf_state["TX"]["gold"]

In [ ]:
word_tfidf_state["TX"].most_common(10)

In [ ]:
# def make_dict_hot_freq_state(state):
#     hot_freq_state = {}
#     hot_word_freq = {}
#     for topic in hot_topics:
#         hot_word_freq[topic] = word_freq_state[state][topic]
#     hot_word_freq["state"] = state
#     hot_freq_state[state] = hot_word_freq
#     return hot_freq_state[state]

def make_dict_hot_tf_idf_state(state):
    hot_tf_idf_state = {}
    hot_word_tf_idf = {}
    for topic in hot_topics:
        hot_word_tf_idf[topic] = word_tfidf_state[state][topic]
    hot_word_tf_idf["state"] = state
    hot_tf_idf_state[state] = hot_word_tf_idf
    return hot_tf_idf_state[state]

In [ ]:
tfidf_list = []
for state in new_state_range:
    hot_freq_state[state] = make_dict_hot_tf_idf_state(state)
    pd_n = pd.DataFrame.from_dict(hot_freq_state[state], orient='index').T
    tfidf_list.append(pd_n)

In [ ]:
df_hot_tf_idf=pd.concat(tfidf_list)
df_hot_tf_idf.head(5)

In [ ]:
def plot_topic_mention_tf_idf(topic):
    freq_list = [word_tfidf_state[state][topic] for state in new_state_range]
    # import plotly.express as px
    fig = px.line(x=new_state_range, y=freq_list, title=topic+' mention TF-IDF by state'
                 ,labels=dict(x="state", y= topic+" Mention TF-IDF", color="Place"))
    fig.show()  


In [ ]:
import plotly.express as px
hot_topics = ["covid", "cancel", "softball", "openingceremony", "skateboarding", "basketball", 
              "swimming", "simone", "gold", "marathon", "volleyball", "closing"]
for topic in hot_topics:
    plot_topic_mention_tf_idf(topic)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pylab import cm
# This lets us do slightly fancier LaTeX-based captions in plots
plt.rc('text', usetex=False)

In [ ]:
county_map = gpd.read_file('UScounties/UScounties.shp')

In [ ]:
county_map.plot()

In [ ]:
# Drop Alaska and Hawaii to just be the continental US
county_map = county_map[(county_map['STATE_NAME'] != 'Alaska') 
                          & (county_map['STATE_NAME'] != 'Hawaii')]

In [ ]:
# This is our fancy projection
projection = "+proj=laea +lat_0=30 +lon_0=-95"
county_map = county_map.to_crs(projection)

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
county_map.plot(ax=ax)

In [ ]:
#append state and convert city to state in map
cities_list = county_map["STATE_NAME"]
states_list = []
for city in cities_list:
    state = us_state_to_abbrev[city]
    states_list.append(state)
county_map['state'] = states_list
county_map.head(5)

In [ ]:
df_hot_freq.head(5)

In [ ]:
df_hot_tf_idf.head(5)

# Join the Data

### Freq Join

In [ ]:
county_tweet_df = pd.merge(county_map, df_hot_freq, on='state', how='inner')
county_tweet_df.head(5)

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
ax.set_title('skateboarding', fontsize=28)
county_tweet_df.plot(ax=ax, column='skateboarding', legend=True,
                                            scheme='quantiles')

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
ax.set_title('basketball', fontsize=28)
county_tweet_df.plot(ax=ax, column='basketball', legend=True,
                                            scheme='quantiles')

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
ax.set_title('volleyball', fontsize=28)
county_tweet_df.plot(ax=ax, column='volleyball', legend=True,
                                            scheme='quantiles')

## TF-IDF Join

In [ ]:
county_tweet_tf_idf_df = pd.merge(county_map, df_hot_tf_idf, on='state', how='inner')
county_tweet_tf_idf_df.head(5)

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
ax.set_title('skateboarding', fontsize=28)
county_tweet_tf_idf_df.plot(ax=ax, column='skateboarding', legend=True,
                                            scheme='quantiles')

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
ax.set_title('basketball', fontsize=28)
county_tweet_tf_idf_df.plot(ax=ax, column='basketball', legend=True,
                                            scheme='quantiles')

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
ax.set_title('volleyball', fontsize=28)
county_tweet_tf_idf_df.plot(ax=ax, column='volleyball', legend=True,
                                            scheme='quantiles')